<a href="https://colab.research.google.com/github/Krithikasrini/Scaler_BusinessCaseStudies/blob/main/Copy_of_Untitled81.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!gdown 1ia46rJItVzQLaSDue5wli1vq4wzmqbRe
!gdown 18IWeyR1d-aOE572lHxyNaOYRrchtzoc2
!gdown 1Y0Jq-LVbfmCTWcueCnwNbkXBslMppOC5

Downloading...
From: https://drive.google.com/uc?id=1ia46rJItVzQLaSDue5wli1vq4wzmqbRe
To: /content/zee-movies.dat
100% 171k/171k [00:00<00:00, 69.3MB/s]
Downloading...
From: https://drive.google.com/uc?id=18IWeyR1d-aOE572lHxyNaOYRrchtzoc2
To: /content/zee-users.dat
100% 134k/134k [00:00<00:00, 59.3MB/s]
Downloading...
From: https://drive.google.com/uc?id=1Y0Jq-LVbfmCTWcueCnwNbkXBslMppOC5
To: /content/zee-ratings.dat
100% 24.6M/24.6M [00:00<00:00, 29.7MB/s]


In [ ]:
import pandas as pd
from datetime import datetime
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.impute import KNNImputer
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split

In [ ]:
movies = pd.read_csv('zee-movies.dat',delimiter='::',engine='python',encoding='latin-1')
users = pd.read_csv('zee-users.dat',delimiter='::',engine='python')
ratings = pd.read_csv('zee-ratings.dat',delimiter='::',engine='python')

In [ ]:
movies

,Movie ID,Title,Genres
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
3878,3948,Meet the Parents (2000),Comedy
3879,3949,Requiem for a Dream (2000),Drama
3880,3950,Tigerland (2000),Drama
3881,3951,Two Family House (2000),Drama


In [ ]:
movies.head(5)

,Movie ID,Title,Genres
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [ ]:
movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3883 entries, 0 to 3882
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Movie ID  3883 non-null   int64 
 1   Title     3883 non-null   object
 2   Genres    3883 non-null   object
dtypes: int64(1), object(2)
memory usage: 91.1+ KB


In [ ]:
movies.nunique()

Movie ID    3883
Title       3883
Genres       301
dtype: int64

In [ ]:
movies.rename(columns={"Movie ID": "MovieID"}, inplace=True)

In [ ]:
ratings.head(5)

,UserID,MovieID,Rating,Timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291



Transforming Movie Genres into Binary Features
We are transforming the data in this manner to prepare it for use in recommendation system.


In [ ]:
m = movies.copy()
m['Genres'] = m['Genres'].str.split('|')
m = m.explode('Genres')
m = m.pivot(index='MovieID', columns='Genres', values='Title')
m = ~m.isna()
m = m.astype(int)
m.head(2)

Genres,Action,Adventure,Animation,Children's,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
MovieID,,,,,,,,,,,,,,,,,,
1,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0


### Extracting Year from Timestamp

 - Extracting the year from the timestamp can indeed serve as a valuable user feature.

In [ ]:
r = ratings.copy()

In [ ]:
r['Release_Year'] = r['Timestamp'].apply(lambda x: datetime.fromtimestamp(x).year)
r.head()

,UserID,MovieID,Rating,Timestamp,Release_Year
0,1,1193,5,978300760,2000
1,1,661,3,978302109,2000
2,1,914,3,978301968,2000
3,1,3408,4,978300275,2000
4,1,2355,5,978824291,2001


In [ ]:
users = pd.read_csv('zee-users.dat',delimiter='::',engine='python')

In [ ]:
users = users.merge(r.groupby('UserID').Rating.mean().reset_index(), on='UserID')
users = users.merge(r.groupby('UserID').Rating.count().reset_index(), on='UserID')
users.head(2)

,UserID,Gender,Age,Occupation,Zip-code,Rating_x,Rating_y
0,1,F,1,10,48067,4.188679,53
1,2,M,56,16,70072,3.713178,129


In [ ]:
#u = users.copy()
#cols = ['Gender','Age','Zip-code']
#u[cols] = u[cols].apply(LabelEncoder().fit_transform)
#u.head()

In [ ]:
#u = u.set_index('UserID')
#u.columns = ['Gender', 'Age', 'Occupation', 'Zip-code', 'Rating_x','Rating_y']

#scaler = StandardScaler()
#u = pd.DataFrame(scaler.fit_transform(u), columns=u.columns, index=u.index)
#u.head(2)

,Gender,Age,Occupation,Zip-code,Rating_x,Rating_y
UserID,,,,,,
1,-1.591927,-1.747373,0.292803,-0.187121,1.131261,-0.584221
2,0.628170,2.251920,1.240822,0.467376,0.024380,-0.189889


In [ ]:
X = ratings[['MovieID', 'UserID', 'Rating']].copy()
X = X.merge(users.reset_index(), on='UserID', how='right')
X = X.merge(m.reset_index(), on='MovieID', how='right')
X.head()

,MovieID,UserID,Rating,index,Gender,Age,Occupation,Zip-code,Rating_x,Rating_y,...,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,1.0,5.0,0.0,F,1.0,10.0,48067,4.188679,53.0,...,0,0,0,0,0,0,0,0,0,0
1,1,6.0,4.0,5.0,F,50.0,9.0,55117,3.901408,71.0,...,0,0,0,0,0,0,0,0,0,0
2,1,8.0,4.0,7.0,M,25.0,12.0,11413,3.884892,139.0,...,0,0,0,0,0,0,0,0,0,0
3,1,9.0,5.0,8.0,M,25.0,17.0,61614,3.735849,106.0,...,0,0,0,0,0,0,0,0,0,0
4,1,10.0,5.0,9.0,F,35.0,1.0,95370,4.114713,401.0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
X.isna().sum()

MovieID          0
UserID         177
Rating         177
index          177
Gender         177
Age            177
Occupation     177
Zip-code       177
Rating_x       177
Rating_y       177
Action           0
Adventure        0
Animation        0
Children's       0
Comedy           0
Crime            0
Documentary      0
Drama            0
Fantasy          0
Film-Noir        0
Horror           0
Musical          0
Mystery          0
Romance          0
Sci-Fi           0
Thriller         0
War              0
Western          0
dtype: int64

**Knn Imputation for missing values**

In [ ]:
X_UserID = X.drop(['Rating','Gender','Age','Occupation','Zip-code','Rating_x','Rating_y'],axis=1)

In [ ]:
X_UserID.columns

Index(['MovieID', 'UserID', 'index', 'Action', 'Adventure', 'Animation',
       'Children's', 'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy',
       'Film-Noir', 'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi',
       'Thriller', 'War', 'Western'],
      dtype='object')

In [ ]:
X.rename(columns={"Children's": "Childrens"}, inplace=True)

In [ ]:
X_UserID = X_UserID.drop(['Adventure','Animation','Childrens','Comedy','Crime','Documentary','Drama','Fantasy','Film-Noir','Horror','Musical','Mystery','Romance','Sci-Fi','Thriller','War','Western','index'],axis=1)

In [ ]:
test_1 = X_UserID[X_UserID.isna().any(axis=1)]
test_1

,MovieID,UserID,Action
25085,51,NaN,1
34063,109,NaN,0
38381,115,NaN,0
40480,143,NaN,0
74693,284,NaN,1
...,...,...,...
947047,3650,NaN,0
968525,3750,NaN,0
984967,3829,NaN,0
987607,3856,NaN,0


In [ ]:
X_UserID = X_UserID.dropna()

In [ ]:
# Scalling the Values except the Traget Variable
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(X_UserID.drop(['UserID'],axis=1))
scaled_features = scaler.transform(X_UserID.drop(['UserID'],axis=1))

In [ ]:
scaled_features

array([[-1.70116023, -0.58874935],
       [-1.70116023, -0.58874935],
       [-1.70116023, -0.58874935],
       ...,
       [ 1.90363475, -0.58874935],
       [ 1.90363475, -0.58874935],
       [ 1.90363475, -0.58874935]])

In [ ]:
scaled_feat_col = ['MovieID','Action']
df_feat = pd.DataFrame(scaled_features,columns=scaled_feat_col)
df_feat.head()

,MovieID,Action
0,-1.70116,-0.588749
1,-1.70116,-0.588749
2,-1.70116,-0.588749
3,-1.70116,-0.588749
4,-1.70116,-0.588749


In [ ]:
# Creating a KNN model with df_feat
Xm = df_feat
y = X_UserID['UserID']

from sklearn.model_selection import train_test_split
X_train, X_valid, y_train, y_valid = train_test_split(Xm, y, test_size=0.33, random_state=99)

from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=1)
knn.fit(X_train,y_train)
pred = knn.predict(X_valid)
pred

array([3091., 1099., 2817., ..., 4578., 4376., 3108.])

In [ ]:
# Calculating the accuracy score
from sklearn.metrics import accuracy_score as acc
print('Accuracy of the model is ',acc(y_valid,pred))

Accuracy of the model is  0.0


In [ ]:
# Creating instance of KNN
knn = KNeighborsClassifier(n_neighbors = 25)

# Fitting the model
knn.fit(X_train, y_train)

# Predicting over the Train Set and calculating accuracy score
valid_predict = knn.predict(X_valid)
k = acc(valid_predict, y_valid)
print('Test Accuracy Score    ', k )

Test Accuracy Score     9.089008661825255e-06


In [ ]:
# Scalling the Values except the Target Variable

scaler = StandardScaler()
scaler.fit(test_1.drop(['UserID'],axis=1))
scaled_features = scaler.transform(test_1.drop(['UserID'],axis=1))
X_test = pd.DataFrame(scaled_features,columns=scaled_feat_col)

In [ ]:
pred1 = knn.predict(X_test)

In [ ]:
pred1 = pd.DataFrame(pred1,columns=['UserID'])

In [ ]:
X

,MovieID,UserID,Rating,index,Gender,Age,Occupation,Zip-code,Rating_x,Rating_y,...,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,1.0,5.0,0.0,F,1.0,10.0,48067,4.188679,53.0,...,0,0,0,0,0,0,0,0,0,0
1,1,6.0,4.0,5.0,F,50.0,9.0,55117,3.901408,71.0,...,0,0,0,0,0,0,0,0,0,0
2,1,8.0,4.0,7.0,M,25.0,12.0,11413,3.884892,139.0,...,0,0,0,0,0,0,0,0,0,0
3,1,9.0,5.0,8.0,M,25.0,17.0,61614,3.735849,106.0,...,0,0,0,0,0,0,0,0,0,0
4,1,10.0,5.0,9.0,F,35.0,1.0,95370,4.114713,401.0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1000381,3952,5812.0,4.0,5811.0,F,25.0,7.0,92120,3.591265,893.0,...,0,0,0,0,0,0,0,1,0,0
1000382,3952,5831.0,3.0,5830.0,M,25.0,1.0,92120,3.677869,1220.0,...,0,0,0,0,0,0,0,1,0,0
1000383,3952,5837.0,4.0,5836.0,M,25.0,7.0,60607,3.895131,534.0,...,0,0,0,0,0,0,0,1,0,0
1000384,3952,5927.0,1.0,5926.0,M,35.0,14.0,10003,3.680556,216.0,...,0,0,0,0,0,0,0,1,0,0


In [ ]:
# Replacing the missing values in Product_category_2 with the predicted values
gen_alt = (alt for alt in pred1['UserID'])
for count, ele in enumerate(X.UserID):
    if not pd.isna(ele): continue
    try:
        X['UserID'][count] = next(gen_alt)
    except StopIteration:
        break

<ipython-input-31-8d2b6fbbe283>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['UserID'][count] = next(gen_alt)


In [ ]:
X.isna().sum()

MovieID          0
UserID           0
Rating         177
index          177
Gender         177
Age            177
Occupation     177
Zip-code       177
Rating_x       177
Rating_y       177
Action           0
Adventure        0
Animation        0
Childrens        0
Comedy           0
Crime            0
Documentary      0
Drama            0
Fantasy          0
Film-Noir        0
Horror           0
Musical          0
Mystery          0
Romance          0
Sci-Fi           0
Thriller         0
War              0
Western          0
dtype: int64

In [ ]:
X

,MovieID,UserID,Action
0,-1.701160,-1.749301,-0.588749
1,-1.701160,-1.746408,-0.588749
2,-1.701160,-1.745251,-0.588749
3,-1.701160,-1.744672,-0.588749
4,-1.701160,-1.744094,-0.588749
...,...,...,...
1000204,1.903635,1.612745,-0.588749
1000205,1.903635,1.623738,-0.588749
1000206,1.903635,1.627209,-0.588749
1000207,1.903635,1.679280,-0.588749


In [ ]:
X_Rating = X.drop(['Gender','Age','Occupation','Zip-code','Rating_x','Rating_y'],axis=1)

In [ ]:
X_Rating = X_Rating.drop(['Adventure','Animation','Childrens','Comedy','Crime','Documentary','Drama','Fantasy','Film-Noir','Horror','Musical','Mystery','Romance','Sci-Fi','Thriller','War','Western','index'],axis=1)

In [ ]:
test_2 = X_Rating[X_Rating.isna().any(axis=1)]
test_2

,MovieID,UserID,Rating,Action
25085,51,36.0,NaN,1
34063,109,193.0,NaN,0
38381,115,355.0,NaN,0
40480,143,432.0,NaN,0
74693,284,292.0,NaN,1
...,...,...,...,...
947047,3650,587.0,NaN,0
968525,3750,178.0,NaN,0
984967,3829,178.0,NaN,0
987607,3856,178.0,NaN,0


In [ ]:
X_Rating = X_Rating.dropna()

In [ ]:
# Scalling the Values except the Traget Variable
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(X_Rating.drop(['Rating'],axis=1))
scaled_features = scaler.transform(X_Rating.drop(['Rating'],axis=1))

In [ ]:
scaled_features

array([[-1.70116023, -1.74930089, -0.58874935],
       [-1.70116023, -1.74640806, -0.58874935],
       [-1.70116023, -1.74525093, -0.58874935],
       ...,
       [ 1.90363475,  1.62720921, -0.58874935],
       [ 1.90363475,  1.67928013, -0.58874935],
       [ 1.90363475,  1.72035831, -0.58874935]])

In [ ]:
scaled_feat_col = ['MovieID','UserID','Action']
df_feat = pd.DataFrame(scaled_features,columns=scaled_feat_col)
df_feat.head()

,MovieID,UserID,Action
0,-1.70116,-1.749301,-0.588749
1,-1.70116,-1.746408,-0.588749
2,-1.70116,-1.745251,-0.588749
3,-1.70116,-1.744672,-0.588749
4,-1.70116,-1.744094,-0.588749


In [ ]:
# Creating a KNN model with df_feat
Xm1 = df_feat
y = X_Rating['Rating']

from sklearn.model_selection import train_test_split
X_train, X_valid, y_train, y_valid = train_test_split(Xm1, y, test_size=0.33, random_state=99)

from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=1)
knn.fit(X_train,y_train)
pred = knn.predict(X_valid)
pred

array([4., 3., 2., ..., 2., 5., 4.])

In [ ]:
# Calculating the accuracy score
from sklearn.metrics import accuracy_score as acc
print('Accuracy of the model is ',acc(y_valid,pred))

Accuracy of the model is  0.31237710902871824


In [ ]:
# Creating instance of KNN
knn = KNeighborsClassifier(n_neighbors = 25)

# Fitting the model
knn.fit(X_train, y_train)

# Predicting over the Train Set and calculating accuracy score
valid_predict = knn.predict(X_valid)
k = acc(valid_predict, y_valid)
print('Test Accuracy Score    ', k )

Test Accuracy Score     0.34759095825418324


In [ ]:
# Scalling the Values except the Target Variable

scaler = StandardScaler()
scaler.fit(test_2.drop(['Rating'],axis=1))
scaled_features = scaler.transform(test_2.drop(['Rating'],axis=1))
X_test = pd.DataFrame(scaled_features,columns=scaled_feat_col)

In [ ]:
pred2 = knn.predict(X_test)

In [ ]:
pred2 = pd.DataFrame(pred2,columns=['Rating'])

In [ ]:
# Replacing the missing values in Product_category_2 with the predicted values
gen_alt = (alt for alt in pred2['Rating'])
for count, ele in enumerate(X.Rating):
    if not pd.isna(ele): continue
    try:
        X['Rating'][count] = next(gen_alt)
    except StopIteration:
        break

<ipython-input-46-16d1023f97d5>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['Rating'][count] = next(gen_alt)


In [ ]:
X.isna().sum()

MovieID          0
UserID           0
Rating           0
index          177
Gender         177
Age            177
Occupation     177
Zip-code       177
Rating_x       177
Rating_y       177
Action           0
Adventure        0
Animation        0
Childrens        0
Comedy           0
Crime            0
Documentary      0
Drama            0
Fantasy          0
Film-Noir        0
Horror           0
Musical          0
Mystery          0
Romance          0
Sci-Fi           0
Thriller         0
War              0
Western          0
dtype: int64

In [ ]:
X_Age = X.drop(['Gender','Occupation','Zip-code','Rating_x','Rating_y'],axis=1)

In [ ]:
X_Age.rename(columns={"Children's": "Childrens"}, inplace=True)

In [ ]:
X_Age = X_Age.drop(['Adventure','Animation','Childrens','Comedy','Crime','Documentary','Drama','Fantasy','Film-Noir','Horror','Musical','Mystery','Romance','Sci-Fi','Thriller','War','Western','index'],axis=1)

In [ ]:
test_3 = X_Age[X_Age.isna().any(axis=1)]
test_3

,MovieID,UserID,Rating,Age,Action
25085,51,36.0,4.0,NaN,1
34063,109,193.0,4.0,NaN,0
38381,115,355.0,4.0,NaN,0
40480,143,432.0,5.0,NaN,0
74693,284,292.0,5.0,NaN,1
...,...,...,...,...,...
947047,3650,587.0,3.0,NaN,0
968525,3750,178.0,4.0,NaN,0
984967,3829,178.0,4.0,NaN,0
987607,3856,178.0,4.0,NaN,0


In [ ]:
X_Age = X_Age.dropna()

In [ ]:
X_Age

,MovieID,UserID,Rating,Age,Action
0,1,1.0,5.0,1.0,0
1,1,6.0,4.0,50.0,0
2,1,8.0,4.0,25.0,0
3,1,9.0,5.0,25.0,0
4,1,10.0,5.0,35.0,0
...,...,...,...,...,...
1000381,3952,5812.0,4.0,25.0,0
1000382,3952,5831.0,3.0,25.0,0
1000383,3952,5837.0,4.0,25.0,0
1000384,3952,5927.0,1.0,35.0,0


In [ ]:
# Scalling the Values except the Traget Variable
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(X_Age.drop(['Age'],axis=1))
scaled_features = scaler.transform(X_Age.drop(['Age'],axis=1))

In [ ]:
scaled_features

array([[-1.70116023, -1.74930089,  1.26974659, -0.58874935],
       [-1.70116023, -1.74640806,  0.37457261, -0.58874935],
       [-1.70116023, -1.74525093,  0.37457261, -0.58874935],
       ...,
       [ 1.90363475,  1.62720921,  0.37457261, -0.58874935],
       [ 1.90363475,  1.67928013, -2.31094931, -0.58874935],
       [ 1.90363475,  1.72035831,  0.37457261, -0.58874935]])

In [ ]:
scaled_feat_col = ['MovieID','UserID','Rating','Action']
df_feat = pd.DataFrame(scaled_features,columns=scaled_feat_col)
df_feat.head()

,MovieID,UserID,Rating,Action
0,-1.70116,-1.749301,1.269747,-0.588749
1,-1.70116,-1.746408,0.374573,-0.588749
2,-1.70116,-1.745251,0.374573,-0.588749
3,-1.70116,-1.744672,1.269747,-0.588749
4,-1.70116,-1.744094,1.269747,-0.588749


In [ ]:
# Creating a KNN model with df_feat
Xm2 = df_feat
y = X_Age['Age']

from sklearn.model_selection import train_test_split
X_train, X_valid, y_train, y_valid = train_test_split(Xm2, y, test_size=0.33, random_state=99)

from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=1)
knn.fit(X_train,y_train)
pred = knn.predict(X_valid)
pred

array([35., 18., 50., ..., 18., 25., 25.])

In [ ]:
# Calculating the accuracy score
from sklearn.metrics import accuracy_score as acc
print('Accuracy of the model is ',acc(y_valid,pred))

Accuracy of the model is  0.388430903841318


In [ ]:
# Creating instance of KNN
knn = KNeighborsClassifier(n_neighbors = 25)

# Fitting the model
knn.fit(X_train, y_train)

# Predicting over the Train Set and calculating accuracy score
valid_predict = knn.predict(X_valid)
k = acc(valid_predict, y_valid)
print('Test Accuracy Score    ', k )

Test Accuracy Score     0.3959626623524172


In [ ]:
# Scalling the Values except the Target Variable

scaler = StandardScaler()
scaler.fit(test_3.drop(['Age'],axis=1))
scaled_features = scaler.transform(test_3.drop(['Age'],axis=1))
X_test = pd.DataFrame(scaled_features,columns=scaled_feat_col)

In [ ]:
pred3 = knn.predict(X_test)

In [ ]:
pred3 = pd.DataFrame(pred3,columns=['Age'])

In [ ]:
# Replacing the missing values in Product_category_2 with the predicted values
gen_alt = (alt for alt in pred3['Age'])
for count, ele in enumerate(X.Age):
    if not pd.isna(ele): continue
    try:
        X['Age'][count] = next(gen_alt)
    except StopIteration:
        break

<ipython-input-63-66c44c9879b2>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['Age'][count] = next(gen_alt)


In [ ]:
X.isna().sum()

MovieID          0
UserID           0
Rating           0
index          177
Gender         177
Age              0
Occupation     177
Zip-code       177
Rating_x       177
Rating_y       177
Action           0
Adventure        0
Animation        0
Childrens        0
Comedy           0
Crime            0
Documentary      0
Drama            0
Fantasy          0
Film-Noir        0
Horror           0
Musical          0
Mystery          0
Romance          0
Sci-Fi           0
Thriller         0
War              0
Western          0
dtype: int64

In [ ]:
X = X.drop(['index'],axis=1)

In [ ]:
X_Gender = X.drop(['Occupation','Zip-code','Rating_x','Rating_y'],axis=1)

In [ ]:
X_Gender = X_Gender.drop(['Adventure','Animation','Childrens','Comedy','Crime','Documentary','Drama','Fantasy','Film-Noir','Horror','Musical','Mystery','Romance','Sci-Fi','Thriller','War','Western'],axis=1)

In [ ]:
test_4 = X_Gender[X_Gender.isna().any(axis=1)]
test_4

,MovieID,UserID,Rating,Gender,Age,Action
25085,51,36.0,4.0,NaN,18.0,1
34063,109,193.0,4.0,NaN,25.0,0
38381,115,355.0,4.0,NaN,25.0,0
40480,143,432.0,5.0,NaN,25.0,0
74693,284,292.0,5.0,NaN,25.0,1
...,...,...,...,...,...,...
947047,3650,587.0,3.0,NaN,25.0,0
968525,3750,178.0,4.0,NaN,25.0,0
984967,3829,178.0,4.0,NaN,25.0,0
987607,3856,178.0,4.0,NaN,25.0,0


In [ ]:
X_Gender = X_Gender.dropna()

In [ ]:
# Scalling the Values except the Traget Variable
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(X_Gender.drop(['Gender'],axis=1))
scaled_features = scaler.transform(X_Gender.drop(['Gender'],axis=1))

In [ ]:
scaled_feat_col = ['MovieID','UserID','Rating','Age','Action']
df_feat = pd.DataFrame(scaled_features,columns=scaled_feat_col)
df_feat.head()

,MovieID,UserID,Rating,Age,Action
0,-1.70116,-1.749301,1.269747,-2.445403,-0.588749
1,-1.70116,-1.746408,0.374573,1.724109,-0.588749
2,-1.70116,-1.745251,0.374573,-0.403193,-0.588749
3,-1.70116,-1.744672,1.269747,-0.403193,-0.588749
4,-1.70116,-1.744094,1.269747,0.447728,-0.588749


In [ ]:
# Creating a KNN model with df_feat
Xm3 = df_feat
y = X_Gender['Gender']

from sklearn.model_selection import train_test_split
X_train, X_valid, y_train, y_valid = train_test_split(Xm3, y, test_size=0.33, random_state=99)

from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=1)
knn.fit(X_train,y_train)
pred = knn.predict(X_valid)
pred

array(['M', 'M', 'M', ..., 'M', 'M', 'F'], dtype=object)

In [ ]:
# Calculating the accuracy score
from sklearn.metrics import accuracy_score as acc
print('Accuracy of the model is ',acc(y_valid,pred))

Accuracy of the model is  0.7551966407023986


In [ ]:
# Scalling the Values except the Target Variable

scaler = StandardScaler()
scaler.fit(test_4.drop(['Gender'],axis=1))
scaled_features = scaler.transform(test_4.drop(['Gender'],axis=1))
X_test = pd.DataFrame(scaled_features,columns=scaled_feat_col)

In [ ]:
# Creating instance of KNN
knn = KNeighborsClassifier(n_neighbors = 25)

# Fitting the model
knn.fit(X_train, y_train)

# Predicting over the Train Set and calculating accuracy score
valid_predict = knn.predict(X_valid)
k = acc(valid_predict, y_valid)
print('Test Accuracy Score    ', k )

Test Accuracy Score     0.7638160505833629


In [ ]:
pred4 = knn.predict(X_test)

In [ ]:
pred4 = pd.DataFrame(pred4,columns=['Gender'])

In [ ]:
# Replacing the missing values in Product_category_2 with the predicted values
gen_alt = (alt for alt in pred4['Gender'])
for count, ele in enumerate(X.Gender):
    if not pd.isna(ele): continue
    try:
        X['Gender'][count] = next(gen_alt)
    except StopIteration:
        break

In [ ]:
X.isna().sum()

MovieID          0
UserID           0
Rating           0
Gender           0
Age              0
Occupation     177
Zip-code       177
Rating_x       177
Rating_y       177
Action           0
Adventure        0
Animation        0
Childrens        0
Comedy           0
Crime            0
Documentary      0
Drama            0
Fantasy          0
Film-Noir        0
Horror           0
Musical          0
Mystery          0
Romance          0
Sci-Fi           0
Thriller         0
War              0
Western          0
dtype: int64

In [ ]:
X.fillna(0)


,MovieID,UserID,Rating,Gender,Age,Occupation,Zip-code,Rating_x,Rating_y,Action,...,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,1.0,5.0,F,1.0,10.0,48067,4.188679,53.0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,6.0,4.0,F,50.0,9.0,55117,3.901408,71.0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,8.0,4.0,M,25.0,12.0,11413,3.884892,139.0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,9.0,5.0,M,25.0,17.0,61614,3.735849,106.0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,10.0,5.0,F,35.0,1.0,95370,4.114713,401.0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1000381,3952,5812.0,4.0,F,25.0,7.0,92120,3.591265,893.0,0,...,0,0,0,0,0,0,0,1,0,0
1000382,3952,5831.0,3.0,M,25.0,1.0,92120,3.677869,1220.0,0,...,0,0,0,0,0,0,0,1,0,0
1000383,3952,5837.0,4.0,M,25.0,7.0,60607,3.895131,534.0,0,...,0,0,0,0,0,0,0,1,0,0
1000384,3952,5927.0,1.0,M,35.0,14.0,10003,3.680556,216.0,0,...,0,0,0,0,0,0,0,1,0,0


In [ ]:
X.isna().sum()

MovieID          0
UserID           0
Rating           0
Gender           0
Age              0
Occupation     177
Zip-code       177
Rating_x       177
Rating_y       177
Action           0
Adventure        0
Animation        0
Childrens        0
Comedy           0
Crime            0
Documentary      0
Drama            0
Fantasy          0
Film-Noir        0
Horror           0
Musical          0
Mystery          0
Romance          0
Sci-Fi           0
Thriller         0
War              0
Western          0
dtype: int64

In [ ]:
X_Occupation = X.drop(['Zip-code','Rating_x','Rating_y'],axis=1)

In [ ]:
X_Occupation = X_Occupation.drop(['Adventure','Animation','Childrens','Comedy','Crime','Documentary','Drama','Fantasy','Film-Noir','Horror','Musical','Mystery','Romance','Sci-Fi','Thriller','War','Western'],axis=1)

In [ ]:
test_5 = X_Occupation[X_Occupation.isna().any(axis=1)]
test_5

,MovieID,UserID,Rating,Gender,Age,Occupation,Action
25085,51,36.0,4.0,M,18.0,NaN,1
34063,109,193.0,4.0,M,25.0,NaN,0
38381,115,355.0,4.0,M,25.0,NaN,0
40480,143,432.0,5.0,M,25.0,NaN,0
74693,284,292.0,5.0,M,25.0,NaN,1
...,...,...,...,...,...,...,...
947047,3650,587.0,3.0,M,25.0,NaN,0
968525,3750,178.0,4.0,M,25.0,NaN,0
984967,3829,178.0,4.0,M,25.0,NaN,0
987607,3856,178.0,4.0,M,25.0,NaN,0


In [ ]:
X_Occupation = X_Occupation.dropna()

In [ ]:
# Scalling the Values except the Traget Variable
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(X_Occupation.drop(['Occupation'],axis=1))
scaled_features = scaler.transform(X_Occupation.drop(['Occupation'],axis=1))

ValueError: ignored

In [ ]:
scaled_feat_col = ['MovieID','UserID','Rating','Gender','Age','Action']
df_feat = pd.DataFrame(scaled_features,columns=scaled_feat_col)
df_feat.head()

In [ ]:
# Creating a KNN model with df_feat
Xm3 = df_feat
y = X_Occupation['Gender']

from sklearn.model_selection import train_test_split
X_train, X_valid, y_train, y_valid = train_test_split(Xm3, y, test_size=0.33, random_state=99)

from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=1)
knn.fit(X_train,y_train)
pred = knn.predict(X_valid)
pred

In [ ]:
# Scalling the Values except the Target Variable

scaler = StandardScaler()
scaler.fit(test_4.drop(['Gender'],axis=1))
scaled_features = scaler.transform(test_4.drop(['Gender'],axis=1))
X_test = pd.DataFrame(scaled_features,columns=scaled_feat_col)

In [ ]:
# Creating instance of KNN
knn = KNeighborsClassifier(n_neighbors = 25)

# Fitting the model
knn.fit(X_train, y_train)

# Predicting over the Train Set and calculating accuracy score
valid_predict = knn.predict(X_valid)
k = acc(valid_predict, y_valid)
print('Test Accuracy Score    ', k )

In [ ]:
pred4 = knn.predict(X_test)

In [ ]:
pred4 = pd.DataFrame(pred4,columns=['Gender'])

In [ ]:
# Replacing the missing values in Product_category_2 with the predicted values
gen_alt = (alt for alt in pred4['Gender'])
for count, ele in enumerate(X.Gender):
    if not pd.isna(ele): continue
    try:
        X['Gender'][count] = next(gen_alt)
    except StopIteration:
        break

In [ ]:
X_Occupation = X.drop(['Occupation','Zip-code','Rating_x','Rating_y'],axis=1)

In [ ]:
X_Occupation = X_Occupation.drop(['Adventure','Animation','Childrens','Comedy','Crime','Documentary','Drama','Fantasy','Film-Noir','Horror','Musical','Mystery','Romance','Sci-Fi','Thriller','War','Western'],axis=1)

In [ ]:
test_4 = X_Occupation[X_Gender.isna().any(axis=1)]
test_4

In [ ]:
X_Gender = X_Gender.dropna()

In [ ]:
# Scalling the Values except the Traget Variable
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(X_Gender.drop(['Gender'],axis=1))
scaled_features = scaler.transform(X_Gender.drop(['Gender'],axis=1))

In [ ]:
scaled_feat_col = ['MovieID','UserID','Rating','Age','Action']
df_feat = pd.DataFrame(scaled_features,columns=scaled_feat_col)
df_feat.head()

In [ ]:
# Creating a KNN model with df_feat
Xm3 = df_feat
y = X_Gender['Gender']

from sklearn.model_selection import train_test_split
X_train, X_valid, y_train, y_valid = train_test_split(Xm3, y, test_size=0.33, random_state=99)

from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=1)
knn.fit(X_train,y_train)
pred = knn.predict(X_valid)
pred

In [ ]:
# Scalling the Values except the Target Variable

scaler = StandardScaler()
scaler.fit(test_4.drop(['Gender'],axis=1))
scaled_features = scaler.transform(test_4.drop(['Gender'],axis=1))
X_test = pd.DataFrame(scaled_features,columns=scaled_feat_col)

In [ ]:
# Creating instance of KNN
knn = KNeighborsClassifier(n_neighbors = 25)

# Fitting the model
knn.fit(X_train, y_train)

# Predicting over the Train Set and calculating accuracy score
valid_predict = knn.predict(X_valid)
k = acc(valid_predict, y_valid)
print('Test Accuracy Score    ', k )

In [ ]:
pred4 = knn.predict(X_test)

In [ ]:
pred4 = pd.DataFrame(pred4,columns=['Gender'])

In [ ]:
# Replacing the missing values in Product_category_2 with the predicted values
gen_alt = (alt for alt in pred4['Gender'])
for count, ele in enumerate(X.Gender):
    if not pd.isna(ele): continue
    try:
        X['Gender'][count] = next(gen_alt)
    except StopIteration:
        break

In [ ]:
X_Gender = X.drop(['Occupation','Zip-code','Rating_x','Rating_y'],axis=1)

In [ ]:
X_Gender = X_Gender.drop(['Adventure','Animation','Childrens','Comedy','Crime','Documentary','Drama','Fantasy','Film-Noir','Horror','Musical','Mystery','Romance','Sci-Fi','Thriller','War','Western'],axis=1)

In [ ]:
test_4 = X_Gender[X_Gender.isna().any(axis=1)]
test_4

In [ ]:
X_Gender = X_Gender.dropna()

In [ ]:
# Scalling the Values except the Traget Variable
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(X_Gender.drop(['Gender'],axis=1))
scaled_features = scaler.transform(X_Gender.drop(['Gender'],axis=1))

In [ ]:
scaled_feat_col = ['MovieID','UserID','Rating','Age','Action']
df_feat = pd.DataFrame(scaled_features,columns=scaled_feat_col)
df_feat.head()

In [ ]:
# Creating a KNN model with df_feat
Xm3 = df_feat
y = X_Gender['Gender']

from sklearn.model_selection import train_test_split
X_train, X_valid, y_train, y_valid = train_test_split(Xm3, y, test_size=0.33, random_state=99)

from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=1)
knn.fit(X_train,y_train)
pred = knn.predict(X_valid)
pred

In [ ]:
# Scalling the Values except the Target Variable

scaler = StandardScaler()
scaler.fit(test_4.drop(['Gender'],axis=1))
scaled_features = scaler.transform(test_4.drop(['Gender'],axis=1))
X_test = pd.DataFrame(scaled_features,columns=scaled_feat_col)

In [ ]:
# Creating instance of KNN
knn = KNeighborsClassifier(n_neighbors = 25)

# Fitting the model
knn.fit(X_train, y_train)

# Predicting over the Train Set and calculating accuracy score
valid_predict = knn.predict(X_valid)
k = acc(valid_predict, y_valid)
print('Test Accuracy Score    ', k )

In [ ]:
pred4 = knn.predict(X_test)

In [ ]:
pred4 = pd.DataFrame(pred4,columns=['Gender'])

In [ ]:
# Replacing the missing values in Product_category_2 with the predicted values
gen_alt = (alt for alt in pred4['Gender'])
for count, ele in enumerate(X.Gender):
    if not pd.isna(ele): continue
    try:
        X['Gender'][count] = next(gen_alt)
    except StopIteration:
        break

In [ ]:
X_Gender = X.drop(['Occupation','Zip-code','Rating_x','Rating_y'],axis=1)

In [ ]:
X_Gender = X_Gender.drop(['Adventure','Animation','Childrens','Comedy','Crime','Documentary','Drama','Fantasy','Film-Noir','Horror','Musical','Mystery','Romance','Sci-Fi','Thriller','War','Western'],axis=1)

In [ ]:
test_4 = X_Gender[X_Gender.isna().any(axis=1)]
test_4

In [ ]:
X_Gender = X_Gender.dropna()

In [ ]:
# Scalling the Values except the Traget Variable
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(X_Gender.drop(['Gender'],axis=1))
scaled_features = scaler.transform(X_Gender.drop(['Gender'],axis=1))

In [ ]:
scaled_feat_col = ['MovieID','UserID','Rating','Age','Action']
df_feat = pd.DataFrame(scaled_features,columns=scaled_feat_col)
df_feat.head()

In [ ]:
# Creating a KNN model with df_feat
Xm3 = df_feat
y = X_Gender['Gender']

from sklearn.model_selection import train_test_split
X_train, X_valid, y_train, y_valid = train_test_split(Xm3, y, test_size=0.33, random_state=99)

from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=1)
knn.fit(X_train,y_train)
pred = knn.predict(X_valid)
pred

In [ ]:
# Scalling the Values except the Target Variable

scaler = StandardScaler()
scaler.fit(test_4.drop(['Gender'],axis=1))
scaled_features = scaler.transform(test_4.drop(['Gender'],axis=1))
X_test = pd.DataFrame(scaled_features,columns=scaled_feat_col)

In [ ]:
# Creating instance of KNN
knn = KNeighborsClassifier(n_neighbors = 25)

# Fitting the model
knn.fit(X_train, y_train)

# Predicting over the Train Set and calculating accuracy score
valid_predict = knn.predict(X_valid)
k = acc(valid_predict, y_valid)
print('Test Accuracy Score    ', k )

In [ ]:
pred4 = knn.predict(X_test)

In [ ]:
pred4 = pd.DataFrame(pred4,columns=['Gender'])

In [ ]:
# Replacing the missing values in Product_category_2 with the predicted values
gen_alt = (alt for alt in pred4['Gender'])
for count, ele in enumerate(X.Gender):
    if not pd.isna(ele): continue
    try:
        X['Gender'][count] = next(gen_alt)
    except StopIteration:
        break

In [ ]:
X = X.dropna()

In [ ]:
X

,MovieID,UserID,Rating,Gender,Age,Occupation,Zip-code,Rating_x,Rating_y,Action,...,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,1.0,5.0,F,1.0,10.0,48067,4.188679,53.0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,6.0,4.0,F,50.0,9.0,55117,3.901408,71.0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,8.0,4.0,M,25.0,12.0,11413,3.884892,139.0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,9.0,5.0,M,25.0,17.0,61614,3.735849,106.0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,10.0,5.0,F,35.0,1.0,95370,4.114713,401.0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1000381,3952,5812.0,4.0,F,25.0,7.0,92120,3.591265,893.0,0,...,0,0,0,0,0,0,0,1,0,0
1000382,3952,5831.0,3.0,M,25.0,1.0,92120,3.677869,1220.0,0,...,0,0,0,0,0,0,0,1,0,0
1000383,3952,5837.0,4.0,M,25.0,7.0,60607,3.895131,534.0,0,...,0,0,0,0,0,0,0,1,0,0
1000384,3952,5927.0,1.0,M,35.0,14.0,10003,3.680556,216.0,0,...,0,0,0,0,0,0,0,1,0,0


In [ ]:
cols = ['Gender','Age','Zip-code']
X[cols] = X[cols].apply(LabelEncoder().fit_transform)
X.head()

,MovieID,UserID,Rating,Gender,Age,Occupation,Zip-code,Rating_x,Rating_y,Action,...,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,1.0,5.0,0,0,10.0,1588,4.188679,53.0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,6.0,4.0,0,5,9.0,1863,3.901408,71.0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,8.0,4.0,1,2,12.0,488,3.884892,139.0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,9.0,5.0,1,2,17.0,2106,3.735849,106.0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,10.0,5.0,0,3,1.0,3188,4.114713,401.0,0,...,0,0,0,0,0,0,0,0,0,0


**Recommendation systems**

In [ ]:
ratings.head()

,UserID,MovieID,Rating,Timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [ ]:
rm = ratings.pivot(index = 'UserID', columns ='MovieID', values = 'Rating').fillna(0)
rm.head()

MovieID,1,2,3,4,5,6,7,8,9,10,...,3943,3944,3945,3946,3947,3948,3949,3950,3951,3952
UserID,,,,,,,,,,,,,,,,,,,,,
1,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
rm.shape

(6040, 3706)

In [ ]:
import numpy as np

In [ ]:
(rm > 0).sum().sum() / (rm.shape[0] * rm.shape[1])

0.044683625622312845

In [ ]:
rm_small = rm.copy()
rm_small = rm_small[rm_small.columns[:100]]
rm_small = rm_small.head(100)

In [ ]:
rm_small.shape

(100, 100)

In [ ]:
K = 2
P = np.random.normal(size=(rm_small.shape[0], K))
Q = np.random.normal(size=(rm_small.shape[1], K))

def matrix_factorization(R, P, Q, K, steps=10000, alpha=0.0002, beta=0.02):
    Q = Q.T
    for step in range(steps): # number of iterations
        for i in range(len(R)): # going over each user
            for j in range(len(R[i])): # going over columns / movies
                if R[i][j] == 0:
                    continue
                eij = R[i][j] - np.dot(P[i,:],Q[:,j])
                for k in range(K):
                    x = P[i][k]
                    P[i][k] += alpha * (2 * eij * Q[k][j]) # - beta * P[i][k])
                    Q[k][j] += alpha * (2 * eij * x) # - beta * Q[k][j])
    return P, Q.T


P_, Q_ =  matrix_factorization(rm_small.values.copy(), P.copy(), Q.copy(), 2)


# Predicted Values vs Actual Ratings
print(np.dot(P_[4], Q_[36]), rm_small.values[4, 36])
print(np.dot(P_[1], Q_[0]), rm_small.values[1, 0])
print(np.dot(P_[1], Q_[2]), rm_small.values[1, 2])
print(np.dot(P_[3], Q_[17]), rm_small.values[3, 17])

-2.7381416133167473 0.0
1.6001903732681102 0.0
0.9840433965093449 0.0
3.2171092678423188 0.0


In [ ]:
from sklearn.metrics import mean_squared_error as mse
rm_ = np.dot(P_, Q_.T)
mse(rm_small.values[rm_small > 0], rm_[rm_small > 0])**0.5

0.5458988518319439

In [ ]:
rm_raw = ratings[['UserID', 'MovieID', 'Rating']].copy()
rm_raw.columns = ['UserId', 'ItemId', 'Rating']  # Lib requires specific column names
rm_raw.head(2)

,UserId,ItemId,Rating
0,1,1193,5
1,1,661,3


In [ ]:
!pip install cmfrec

  Using cached cmfrec-3.5.1.post7.tar.gz (268 kB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached findblas-0.1.22-py3-none-any.whl
  Created wheel for cmfrec: filename=cmfrec-3.5.1.post7-cp310-cp310-linux_x86_64.whl size=5830878 sha256=69395c1f21fd057e6e14d78e8325b2cb297085ef53aad425492d1b143178d062
  Stored in directory: /root/.cache/pip/wheels/10/cf/c1/3b05df5e49ce0c5ed2e8fe63f377c6c2c23a63c8b327d8f734
Successfully built cmfrec


In [ ]:
from cmfrec import CMF

model = CMF(method="als", k=2, lambda_=0.1, user_bias=False, item_bias=False, verbose=False)
model.fit(rm_raw)

Collective matrix factorization model
(explicit-feedback variant)


In [ ]:
model.A_.shape, model.B_.shape

((6040, 2), (3706, 2))

In [ ]:
rm_raw.Rating.mean(), model.glob_mean_

(3.581564453029317, 3.581564426422119)

In [ ]:
rm__ = np.dot(model.A_, model.B_.T) + model.glob_mean_
mse(rm.values[rm > 0], rm__[rm > 0])**0.5

1.3043536679938734

In [ ]:
top_items = model.topN(user=1, n=10)
movies.loc[movies.MovieID.isin(top_items)]

,MovieID,Title,Genres
638,643,Peanuts - Die Bank zahlt alles (1996),Comedy
883,895,Venice/Venice (1992),Drama
1397,1421,Grateful Dead (1995),Documentary
2754,2823,"Spiders, The (Die Spinnen, 1. Teil: Der Golden...",Action|Drama
2842,2911,"Grandfather, The (El Abuelo) (1998)",Drama
3264,3333,"Killing of Sister George, The (1968)",Drama
3311,3380,Railroaded! (1947),Film-Noir
3462,3531,All the Vermeers in New York (1990),Comedy|Drama|Romance
3748,3818,Pot O' Gold (1941),Comedy|Musical
3822,3892,Anatomy (Anatomie) (2000),Horror
